### 1.Introduction where you discuss the business problem and who would be interested in this project.

    Consider if you want to open a new cinema in Hong Kong, how would you choose the location?
    let's look at the factors we care about, first it should be convenient，Customers could walk to cinema within 5 minutes from public transport station.and if there are some good restaurants and trading aera around would be better.

### 2.Data where you describe the data that will be used to solve the problem and the source of the data.

#### 1. Geographic coordinate of Hong Kong cinemas

I need to **compare 5 possible locations with current cinemas** in Hong Kong. Therefore, I need to find a list of Hong Kong cinema and cinemas' geographic coordinates. Luckily, I can find the list and coordinates from the website https://hkmovie6.com/cinema .

In [1]:
import json
import pandas as pd

In [2]:
!wget -O hk_cinema_list.json https://hkmovie6.com/api/cinemas/lists

--2019-08-01 14:31:28--  https://hkmovie6.com/api/cinemas/lists
Resolving hkmovie6.com (hkmovie6.com)... 104.31.67.1, 104.31.66.1, 2606:4700:30::681f:4301, ...
Connecting to hkmovie6.com (hkmovie6.com)|104.31.67.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘hk_cinema_list.json’

    [ <=>                                   ] 56,711       347KB/s   in 0.2s   

2019-08-01 14:31:29 (347 KB/s) - ‘hk_cinema_list.json’ saved [56711]



In [3]:
cinemas_json = None
with open('hk_cinema_list.json', 'r') as f:
    cinemas_json = json.load(f)
    
cinemas = []
for data in cinemas_json['data']:
    cinemas.append({
        'Name': data['name'],
        'Address': data['address'],
        'Latitude': data['lat'],
        'Longitude': data['lon']
    })
df_cinemas = pd.DataFrame(cinemas, columns=['Name','Address','Latitude','Longitude'])

In [4]:
print('There are {} cinemas in Hong Kong'.format(len(df_cinemas)))

There are 71 cinemas in Hong Kong


In [5]:
df_cinemas.head()

,Name,Address,Latitude,Longitude
0,Emperor Cinemas - Entertainment Building,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
1,Emperor Cinemas - Ma On Shan,"L2, MOSTown, Sai Sha Road, Ma On Shan, N.T.",22.424120,114.230957
2,Emperor Cinemas - Tuen Mun,"3/F, New Town Commercial Arcade, 2 Tuen Lee St...",22.390776,113.975983
3,The Coronet @ Emperor Cinemas - Entertainment ...,"3/F, Emperor Cinemas Entertainment Building, 3...",22.281453,114.154230
4,Festival Grand Cinema,"Level UG, Festival Walk, 80 Tat Chee Avenue, K...",22.337882,114.174325


#### 2. Geographic coordinates of 5 possible cinema addresses
I also need to know the geographic coordinates of 5 possible cinemas. I can use Google Map API to find this information

In [6]:
possible_locations = [
    { 'Location': 'L1', 'Address': 'Sau Mau Ping Shopping Centre, Sau Mau Ping'},
    { 'Location': 'L2', 'Address': 'Tuen Mun Ferry, Tuen Mun'},
    { 'Location': 'L3', 'Address': 'Un Chau Shopping Centre, Cheung Sha Wan'},
    { 'Location': 'L4', 'Address': 'Prosperity Millennia Plaza, North Point'},
    { 'Location': 'L5', 'Address': 'Tsuen Fung Centre Shopping Arcade, Tsuen Wan'},
]

In [17]:
!pip install -U googlemaps

Requirement already up-to-date: googlemaps in /opt/conda/envs/Python36/lib/python3.6/site-packages (3.0.2)


In [10]:
google_act = None
with open('google_map_act.json', 'r') as f:
    google_act = json.load(f)
    
GOOGLE_MAP_API_KEY = google_act['api_key']    

import googlemaps
gmaps = googlemaps.Client(key=GOOGLE_MAP_API_KEY)

FileNotFoundError: [Errno 2] No such file or directory: 'google_map_act.json'

In [11]:
def getLatLng(address):
    latlnt = gmaps.geocode('{}, Hong Kong'.format(address))
    return (latlnt[0]['geometry']['location']['lat'], latlnt[0]['geometry']['location']['lng'])

Dataframe of 5 possible locations with geographic coordinates information

In [12]:
for loc in possible_locations:        
    (lat, lng) = getLatLng(loc['Address'])
    loc['Latitude'] = lat
    loc['Longitude'] = lng
    
df_possible_locations = pd.DataFrame(possible_locations, columns=['Location', 'Address', 'Latitude', 'Longitude'])
df_possible_locations

NameError: name 'gmaps' is not defined

#### 3. Favorite cinema list

The favorite cinema list is an important information that I can **use it as profile to select the best location**.  
Stakeholder further explains that the rating is range of 1.0 (worst) to 5.0 (best) values

In [18]:
boss_favorite = [
    {'Name': 'Boradway Circuit - MONGKONG', 'Rating': 4.5},
    {'Name': 'Boradway Circuit - The ONE', 'Rating': 4.5},
    {'Name': 'Grand Ocean', 'Rating': 4.3},
    {'Name': 'The Grand Cinema', 'Rating': 3.4},
    {'Name': 'AMC Pacific Place', 'Rating': 2.3},
    {'Name': 'UA IMAX @ Airport', 'Rating': 1.5},
]

df_boss_favorite = pd.DataFrame(boss_favorite, columns=['Name','Rating'])
df_boss_favorite

,Name,Rating
0,Boradway Circuit - MONGKONG,4.5
1,Boradway Circuit - The ONE,4.5
2,Grand Ocean,4.3
3,The Grand Cinema,3.4
4,AMC Pacific Place,2.3
5,UA IMAX @ Airport,1.5


#### 4. Eating, Shopping and Public transportation facility around cinema
The recommended cinema location needs to have many eating and shopping venues nearby. Convenient public transport is also required.  
I can use FourSquare API to find these venues around the location. 

5 minutes walking distance is about 500m. I think it is the suitable distance to search nearby venues.

However, the API provides maximum 50 results only, so it is better to search venues by category. Following categories will be used for finding the target venues. Full list of categories: https://developer.foursquare.com/docs/resources/categories

In [19]:
cinema = df_cinemas.loc[0]

In [20]:
print('Use the first cinema "{}" in the list as example to explore venues nearyby'.format(cinema['Name']))

Use the first cinema "Emperor Cinemas - Entertainment Building" in the list as example to explore venues nearyby


In [21]:
fs_categories = {
    'Food': '4d4b7105d754a06374d81259',
    'Shop & Service': '4d4b7105d754a06378d81259',
    'Bus Stop': '52f2ab2ebcbc57f1066b8b4f',
    'Metro Station': '4bf58dd8d48988d1fd931735',
    'Nightlife Spot': '4d4b7105d754a06376d81259',
    'Arts & Entertainment': '4d4b7104d754a06370d81259'
}

In [22]:
!pip install foursquare

  Stored in directory: /home/dsxuser/.cache/pip/wheels/50/82/db/97c078881efb89f4e89560487926fbadfff87a40e5fe41f6fb
Successfully built foursquare


In [23]:
fs_act = None
with open('fs_act.json') as json_data:
    fs_act = json.load(json_data)

FileNotFoundError: [Errno 2] No such file or directory: 'fs_act.json'

In [24]:
import foursquare
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
fs = foursquare.Foursquare(client_id=fs_act['client_id'], client_secret=fs_act['client_secret'])

TypeError: 'NoneType' object is not subscriptable

In [25]:
RADIUS = 500

In [26]:
ef venues_nearby(latitude, longitude, category):    
    results = fs.venues.search(
        params = {
            'query': category, 
            'll': '{},{}'.format(latitude, longitude),
            'radius': RADIUS,
            'categoryId': fs_categories[category]
        }
    )    
    df = json_normalize(results['venues'])
    cols = ['Name','Latitude','Longitude','Tips','Users','Visits']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng','stats.tipCount','stats.usersCount','stats.visitsCount']]
        df.columns = cols
    print('{} "{}" venues are found within {}m of location'.format(len(df), category, RADIUS))
    return df

SyntaxError: invalid syntax (<ipython-input-26-bc70209e8e86>, line 1)

Find number of MTR station around the cinema

In [27]:
venues_nearby(cinema['Latitude'], cinema['Longitude'], 'Metro Station').head()

NameError: name 'venues_nearby' is not defined